# The first prototype of the sensitivity analysis workflow. 

The workflow is designed to run from the inside of the docker container and assumes a specific folder layout.  

Authors: Tobey Carman and Elchin Jafarov

## Working with the docker 
Assuming that docker was successfully installed, navigate to your local dvmdostem folder:

1. Strat the containers <br/>
`$ docker compose up -d` <br/>
2. Enter to the container <br/>
`$ docker compose exec dvmdostem-run bash` <br/>
3. Start jupyter notebook inside the /work folder <br/>
`$ jupyter notebook --ip 0.0.0.0 --no-browser --allow-root` <br/>
4. Copy the url into your browser. <br/>
5. When done. Shut down container <br/>
`$ docker compose down` <br/>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import output_utils as ou
import param_util as pu
import os
import subprocess

import Sensitivity


In [3]:
param_specs = [
    {'name':'cmax', 'cmtnum':4, 'pftnum':3, 'bounds':[100,700],'enabled':True },
    {'name':'rhq10', 'cmtnum':4, 'pftnum':None, 'bounds':[0.1,5],'enabled':True },
    {'name':'micbnup', 'cmtnum':4, 'pftnum':None, 'bounds':[0.1,10],'enabled':True },
]

In [5]:
def generate_sample_matrix(pspecs, N, method='uniform'):
    
    if not method == 'uniform':
        raise RuntimeError("Not implemented yet!")
    
    sample_matrix = {}
    for i, p in enumerate(filter(lambda x: x['enabled'], param_specs)):
        samples = np.linspace(p['bounds'][0], p['bounds'][1], N)
        sample_matrix[p['name']] = samples

    return pd.DataFrame(sample_matrix)

In [7]:
sample_matrix = generate_sample_matrix(param_specs, 10)
sample_matrix

,cmax,rhq10,micbnup
0,100.000000,0.100000,0.1
1,166.666667,0.644444,1.2
2,233.333333,1.188889,2.3
3,300.000000,1.733333,3.4
4,366.666667,2.277778,4.5
5,433.333333,2.822222,5.6
6,500.000000,3.366667,6.7
7,566.666667,3.911111,7.8
8,633.333333,4.455556,8.9
9,700.000000,5.000000,10.0


In [10]:
driver = Sensitivity.SensitivityDriver(param_specs, sample_matrix)

In [11]:
driver.setup_multi()

[SA:setup] /work/scripts/setup_working_directory.py --input-data-path /data/input-catalog/cru-ts40_ar5_rcp85_ncar-ccsm4_CALM_Toolik_LTER_10x10/ /data/workflows/sensitivity_analysis/sample_000000000

[SA:setup] /work/scripts/runmask-util.py --reset --yx 0 0 /data/workflows/sensitivity_analysis/sample_000000000/run-mask.nc

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000000/config/output_spec.csv --on GPP m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000000/config/output_spec.csv --on VEGC m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000000/config/output_spec.csv --on CMTNUM y

Using CMT0 as reference...
Found tag: cmax  Desc:  
Found tag: nmax  Desc:  
Found tag: cfall(0)  Desc:   leave
Found tag: cfall(1)  Desc:   stem
Found tag: cfall(2)  Desc:   root
Found tag: nfall(0)  Desc:   leave
Found tag: nfall(1)  Desc:   stem
Found tag: nfall(2)  Desc:


[SA:setup] /work/scripts/runmask-util.py --reset --yx 0 0 /data/workflows/sensitivity_analysis/sample_000000003/run-mask.nc

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000003/config/output_spec.csv --on GPP m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000003/config/output_spec.csv --on VEGC m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000003/config/output_spec.csv --on CMTNUM y

Using CMT0 as reference...
Found tag: cmax  Desc:  
Found tag: nmax  Desc:  
Found tag: cfall(0)  Desc:   leave
Found tag: cfall(1)  Desc:   stem
Found tag: cfall(2)  Desc:   root
Found tag: nfall(0)  Desc:   leave
Found tag: nfall(1)  Desc:   stem
Found tag: nfall(2)  Desc:   root
Found tag: kra  Desc:      coeff(0) in maintenance resp.
Found tag: krb(0)  Desc:   coeff(1) in maintenance resp.(leave)
Found tag: krb(1)  Desc:   coeff(1) in maintenance resp.(stem)
Found 


[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000006/config/output_spec.csv --on GPP m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000006/config/output_spec.csv --on VEGC m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000006/config/output_spec.csv --on CMTNUM y

Using CMT0 as reference...
Found tag: cmax  Desc:  
Found tag: nmax  Desc:  
Found tag: cfall(0)  Desc:   leave
Found tag: cfall(1)  Desc:   stem
Found tag: cfall(2)  Desc:   root
Found tag: nfall(0)  Desc:   leave
Found tag: nfall(1)  Desc:   stem
Found tag: nfall(2)  Desc:   root
Found tag: kra  Desc:      coeff(0) in maintenance resp.
Found tag: krb(0)  Desc:   coeff(1) in maintenance resp.(leave)
Found tag: krb(1)  Desc:   coeff(1) in maintenance resp.(stem)
Found tag: krb(2)  Desc:   coeff(1) in maintenance resp.(root)
Found tag: frg  Desc:   fraction of available NPP (GPP after rm) for


[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000009/config/output_spec.csv --on VEGC m p

[SA:setup] /work/scripts/outspec_utils.py /data/workflows/sensitivity_analysis/sample_000000009/config/output_spec.csv --on CMTNUM y

Using CMT0 as reference...
Found tag: cmax  Desc:  
Found tag: nmax  Desc:  
Found tag: cfall(0)  Desc:   leave
Found tag: cfall(1)  Desc:   stem
Found tag: cfall(2)  Desc:   root
Found tag: nfall(0)  Desc:   leave
Found tag: nfall(1)  Desc:   stem
Found tag: nfall(2)  Desc:   root
Found tag: kra  Desc:      coeff(0) in maintenance resp.
Found tag: krb(0)  Desc:   coeff(1) in maintenance resp.(leave)
Found tag: krb(1)  Desc:   coeff(1) in maintenance resp.(stem)
Found tag: krb(2)  Desc:   coeff(1) in maintenance resp.(root)
Found tag: frg  Desc:   fraction of available NPP (GPP after rm) for growth resp.
Found tag: micbnup  Desc:   parameter for soil microbial immobialization of N
Found tag: kdcrawc  Desc:   raw-material 

In [12]:
driver.run_all_samples()

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000000/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000001/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000002/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000003/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000004/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4 --ctrl-file /data/workflows/sensitivity_analysis/sample_000000005/config/config.js

[SA:run] /work/dvmdostem -p 5 -e 5 -s 5 -t 5 -n 5 -l err --force-cmt 4

In [13]:
driver.collect_outputs()



In [14]:
import glob
file_list = glob.glob('/data/workflows/sensitivity_analysis/**/*sensitivity.csv', recursive=True)
df = pd.concat( map(pd.read_csv, file_list), ignore_index=True)
df = df.sort_values('p_cmax')
corr = df.corr()





In [16]:
# Sets initial values from the parameters directory - works better when used with 
# Sensitivity.setup_single() 
# Not sure how to use this yet...
for param in param_specs:
    pfile = pu.which_file(os.path.join(x2.work_dir, "parameters"), param['name'])
    data = pu.get_CMT_datablock(pfile, param['cmtnum'])
    data_dict = pu.cmtdatablock2dict(data)
    if param['pftnum'] is not None:
        pftkey = 'pft{}'.format(param['pftnum'])
        #print(param['name'], data_dict[pftkey][param['name']])
        param['initial_value'] = data_dict[pftkey][param['name']]
    else:
        #print(param['name'], data_dict[param['name']])
        param['initial_value'] = data_dict[param['name']]
    print(param)

